In [1]:
import pyspark
from pyspark.sql import SparkSession, types

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 20:34:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/03 20:34:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


want to upload the original parquet files using the below command:
```sh
gsutil -m cp -r ./pq/ gs://dezoomcamp-tmp/pq
```
`-m` specifies multithreaded operation <br>
`-r` is recursive, I believe.

In [3]:
!gsutil -m cp -r ./data/pq/ gs://dezoomcamp-tmp/pq

Copying file://./data/pq/green/2020/03/part-00002-5bdf1a52-3b86-4ff3-81cf-ce3ace4903b0-c000.snappy.parquet [Content-Type=application/octet-stream]...
Copying file://./data/pq/green/2020/03/.part-00000-5bdf1a52-3b86-4ff3-81cf-ce3ace4903b0-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://./data/pq/green/2020/03/.part-00001-5bdf1a52-3b86-4ff3-81cf-ce3ace4903b0-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://./data/pq/green/2020/03/.part-00003-5bdf1a52-3b86-4ff3-81cf-ce3ace4903b0-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://./data/pq/green/2020/03/._SUCCESS.crc [Content-Type=application/octet-stream]...
Copying file://./data/pq/green/2020/09/.part-00003-0e80b43f-d3d7-4ffe-893b-a92ec40be912-c000.snappy.parquet.crc [Content-Type=application/octet-stream]...
Copying file://./data/pq/green/2020/09/_SUCCESS [Content-Type=application/octet-stream]...
Copying file://./data/pq/green/2020/09/part

Now that the data is in place, we'll need to include a connector that can parse `gs://` addresses, i.e., the Cloud Storage connector for Hadoop 3.x.

In [5]:
!mkdir lib

In [11]:
!cd lib && gsutil cp gs://hadoop-lib/gcs/gcs-connector-hadoop3-2.2.5.jar gcs-connector-hadoop3-2.2.5.jar

Copying gs://hadoop-lib/gcs/gcs-connector-hadoop3-2.2.5.jar...
\ [1 files][ 30.1 MiB/ 30.1 MiB]                                                
Operation completed over 1 objects/30.1 MiB.                                     


In [83]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [84]:
creds_loc = '/home/sam/data-engineering-zoomcamp-notes/google_credentials.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('local_test') \
    .set("spark.jars", "/home/sam/spark/spark-3.3.2-bin-hadoop3/jars/gcs-connector-hadoop3-latest.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", creds_loc)

In [85]:
#sc = SparkContext.getOrCreate(conf=conf)
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", creds_loc)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

In [86]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [87]:
df_green = spark.read.parquet('gs://dezoomcamp-tmp/pq/green/*/*/')

23/03/03 22:22:28 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: gs://dezoomcamp-tmp/pq/green/*/*/.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.Data

Py4JJavaError: An error occurred while calling o1153.parquet.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:752)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGl

In [81]:
spark.stop()
sc.stop()

/home/sam/spark/spark-3.3.2-bin-hadoop3/jars/gcs-connector-hadoop3-latest.jar